<h1 style="text-align:center">Reconhecimento de Padrões</h1>

<p><b>Resumo</b>: Em <i>Machine Learning</i>, existem categorias em relação ao tipo do problema a ser resolvido. Neste trabalho será utilizado apenas dois, o de Regressão e de Classificação. A primeira parte deste trabalho será para analisar os modelos de Classificação e determinar qual deles tem o melhor desempenho em relação a acurácia para a base de dados (nome da bd), a saber, os modelos serão:</p>
    <ul>
        <li><b>Naivy Bayes</b></li>
        <li><b>Regressão Logística</b></li>
        <li><b>Kernel SVM</b></li>
        <li><b>SVM Linear</b></li>
    </ul>

<p>A segunda parte do projeto é analisar qual modelo de Regressão mais se aproxima gráficamente da função original. Os modelos que serão utilizados são:</p>
    <ul>
        <li><b>Regressão Linear</b></li>
        <li><b>Regressão Linear com Kernel RBF</b></li>
    </ul>

<h3 style="text-align:center;">Parte 1 - Preparando dados para serem treinados</h3>

In [1]:
from cvxopt import matrix
from cvxopt import spmatrix
from cvxopt.solvers import qp 
from cvxopt import solvers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time

ImportError: DLL load failed: Não foi possível encontrar o módulo especificado.

In [ ]:
solvers.options["show_progress"] = False

<h5>Métodos auxiliares</h5>

In [ ]:
def class_encode(specie,column,values_y):
    if column in specie:
        return values_y[1]
    else:
        return values_y[0]

In [ ]:
def normalize(data):
    for col in data.columns:
        min = np.min(data[col])
        max = np.max(data[col])
        data[col] = [(data.at[i,col] - min)/(max-min)
                        for i in range(len(data))]
    return data


In [ ]:
def train_test(data, class_d):
    train_x, test_x = _split_data_(data)
    train_d = class_d.iloc[train_x.index]
    test_d = class_d.drop(train_d.index)
    return train_x,test_x,train_d,test_d

def _split_data_(data):
    index_random = _random_index_(data,0.8)
    return data.loc[index_random],data.drop(index_random)

def _random_index_(data,count):
    random_count = int(len(data) * count)
    return np.random.choice(data.index,random_count,replace = False)


In [ ]:
def metrics(modelo, num_realizacoes, db, class_db, specie, list_y):
    inicio_tempo = time.time()
    species = class_db.apply(class_encode,column=specie,values_y=list_y)
    lista_acuracia = []
    for realizacao in range(num_realizacoes):
        train_x, test_x, train_y, test_y = train_test(db, species)
        modelo.train(train_x,train_y)
        list_y = [modelo.predict(list(linha)) for indice, linha in test_x.iterrows()]
        lista_acuracia += [np.mean(list_y == test_y)]
    print("Toda o processo durou: {:.3f}s".format(time.time() - inicio_tempo))
    return np.mean(lista_acuracia)

In [ ]:
def test_model(modelo, db, class_db, specie, list_y):
    species = class_db.apply(class_encode,column=specie,values_y=list_y)
    train_x, test_x, train_y, test_y = train_test(db, species)
    modelo.train(train_x,train_y)
    teste_1 = modelo.predict(db.loc[0])
    teste_2 = modelo.predict(db.loc[51])
    teste_3 = modelo.predict(db.loc[100])
    return "valor da setosa = {}, valor da versicolor = {}, valor da virginica = {}".format(teste_1,teste_2,teste_3)

<h5>Obtendo os dados</h5>

In [ ]:
iris_db = pd.read_csv("iris.csv")
iris_db.head()

<h5>Pré-processamento dos dados</h5>

In [ ]:
species = iris_db.species
iris_db = iris_db.drop(['species'],axis=1)

In [ ]:
iris_db.head()

In [ ]:
species.head()

In [ ]:
iris_db = normalize(iris_db)
iris_db

<h3 style="text-align:center">Modelos de Classificação</h3>
<h5>Naivy Bayes</h5>
<p>O primeiro modelo de Classificação que será analisado é o Naivy Bayes. A seguir está a sua implementação:</p>

In [ ]:
class NaivyBayesGaussian:
    __slots__ = ['probabilidade_y','lista_desvio_padrao','lista_media']
    
    def __init__(self):
        self.probabilidade_y = [0,0]
        self.lista_desvio_padrao = []
        self.lista_media = []        
            
    def train(self, db, class_db):
        self.lista_desvio_padrao = np.zeros((2, db.shape[1]))
        self.lista_media = np.zeros((2, db.shape[1]))
        
        for y in range(2):
            self.probabilidade_y[y] = len(class_db.loc[class_db == y])
            index_db = class_db.loc[class_db == y].index
            for num_column in range(len(db.columns)):
                column = db.columns[num_column]
                self.lista_desvio_padrao[y][num_column] = np.std(db[column].loc[index_db])
                self.lista_media[y][num_column] = np.mean(db[column].loc[index_db])        
       
    def probabildade_x_y(self,x, y):
        resultado = 1
        for num_column in range(len(x)):
            std = self.lista_desvio_padrao[y][num_column]
            mean = self.lista_media[y][num_column]
            resultado *= ((1/(math.sqrt(2*math.pi)*std))*(math.exp((-(x[num_column]-mean)**2)/(2*(std**2)))))
        return resultado
    
    def arg_max(self,lista_probabilidades):
        for i in range(len(lista_probabilidades)):
            if lista_probabilidades[i] == np.max(lista_probabilidades):
                return i
    
    def predict(self,x):
        lista_probabilidades = []
        for y in range(len(self.probabilidade_y)):
            lista_probabilidades += [self.probabilidade_y[y] * self.probabildade_x_y(x,y)]
        return self.arg_max(lista_probabilidades)

In [ ]:
naivy = NaivyBayesGaussian()

In [ ]:
test_model(naivy, iris_db, species, "virginica", [0,1])

In [ ]:
metrics(naivy, 20, iris_db, species, "setosa", [0,1])

<h5>Regressão Logistica</h5>

In [ ]:
class RegressaoLogistica:
    _slots__ = ['w']
    
    def __init__(self):
        self.w = []
    
    def train(self, db, class_db, epocas=100, taxa_de_aprendizagem=0.001):
        self.w = np.zeros((1, db.shape[1] + 1))
        
        for epoca in range(epocas):
            gradiente = np.zeros((1, db.shape[1] + 1))
            for indice, linha in db.iterrows():
                x = np.array(list(linha) + [-1])
                y = class_db.loc[indice]
                gradiente += ((y*x)/(1+np.exp(y * np.dot(x, self.w.T))))
            valor_w_antigo = self.w
            self.w += taxa_de_aprendizagem*gradiente 
    
    def arg_max(self, probabilidade_menos_1, probabilidade_1):
        return -1 if probabilidade_menos_1 > probabilidade_1 else 1
            
    def predict(self, linha):
        x = np.array(list(linha) + [-1])
        
        probabilidade_menos_1 = (1/(1+np.exp(np.dot(x, self.w.T))))
        probabilidade_1 = (1/(1+np.exp(-np.dot(x, self.w.T))))
        return self.arg_max(probabilidade_menos_1,probabilidade_1)
        

In [ ]:
rg = RegressaoLogistica()

In [ ]:
test_model(rg,iris_db,species, "setosa", [-1,1])

In [ ]:
metrics(rg, 20, iris_db, species, "setosa", [-1,1])

<h5>SVM Linear</h5

In [ ]:
class SVMLinear:
    __slots__ = ['w','p','q','g','h']
    
    def __init__(self):
        self.w = []
        self.p = []
        self.q = []
        self.g = []
        self.h = []
        
    def initParams(self, db, class_db):
        self.w = []
        
        matriz_identidade = np.identity(db.shape[1])
        DIMENSAO = db.shape[0] + db.shape[1] + 1 
        matriz_p = np.zeros((DIMENSAO, DIMENSAO))
        matriz_p[:matriz_identidade.shape[0], :matriz_identidade.shape[1]] = matriz_identidade + matriz_p[:matriz_identidade.shape[0], :matriz_identidade.shape[1]]
        self.p = matrix(matriz_p, tc="d")
        
        matriz_aux_q = np.ones((db.shape[0],1))
        matriz_q = np.zeros((DIMENSAO,1))
        matriz_q[db.shape[1] + 1:,:] = matriz_aux_q + matriz_q[db.shape[1] + 1:,:]
        self.q = matrix(matriz_q)
        
        dados_com_classes = db.join(class_db)
        lista_g = []
        for indice,linha in dados_com_classes.iterrows():
            lista_g += [self.getG(linha, class_db.loc[indice])]
        
        matriz_aux_g = np.array(lista_g)
        matriz_identidade_g = np.identity(db.shape[0]) * (-1)
        DIMENSAO_2 = 2*db.shape[0]
        matriz_g = np.zeros((DIMENSAO_2, DIMENSAO))
        matriz_g[:db.shape[0],:db.shape[1]+1] = matriz_aux_g + matriz_g[:db.shape[0],:db.shape[1]+1]
        matriz_g[:db.shape[0],db.shape[1]+1:] = matriz_identidade_g + matriz_g[:db.shape[0],db.shape[1]+1:]
        matriz_g[db.shape[0]:,db.shape[1]+1:] = matriz_identidade_g + matriz_g[db.shape[0]:,db.shape[1]+1:]
                   
        self.g = matrix(matriz_g, tc="d")
                                                                        
        matriz_h = np.zeros((DIMENSAO_2,1))
        matriz_h[:db.shape[0],:] = np.ones((db.shape[0],1)) * (-1)
        self.h = matrix(matriz_h) 
        
    def getG(self,linha, y):
        x = np.array(list(linha[:-1]) + [1])
        return (x * y).T * -1
    
    def train(self, db, class_db):
        self.initParams(db, class_db)
        s = qp(self.p,self.q,self.g, self.h)
        self.w = s['x']

    def predict(self, linha):
        x = np.array(list(linha) + [1])
        u = np.dot(x,self.w[:x.shape[0]])
        return 1 if u > 0 else -1

In [ ]:
svm = SVMLinear()

In [ ]:
test_model(svm, iris_db, species, "setosa", [-1,1])

In [2]:
metrics(svm, 20, iris_db, species, "virginica", [-1,1])

NameError: name 'metrics' is not defined

<h5>Kernel SVM</h5>

In [ ]:
class KernelSVM:
    __slots__ = ['alfa','p','q','g','h','y', 'x', 'beta']
    
    def __init__(self):
        self.alfa = []
        self.p = []
        self.q = []
        self.g = []
        self.h = []
        self.x = []
        self.y = []
        self.beta = 0
        
    def initParams(self, db, class_db, beta):
        self.beta = beta
        self.alfa = []
        x = np.concatenate((db, np.ones((db.shape[0],1)) ), axis=1)
        new_db = pd.DataFrame(x)
        self.x = new_db
        self.y = class_db
        
        kernel = []
        for indice_i, linha_i in self.x.iterrows():
            linha_kernel = []
            for indice_j, linha_j in self.x.iterrows():
                y_1 = class_db.iloc[indice_i]
                y_2 = class_db.iloc[indice_j]
                linha_kernel += [self.kernel_rbf(linha_i, linha_j, beta) * y_1 * y_2 ]
            kernel += [linha_kernel]
        
        
        matriz_p = np.array(kernel)
        self.p = matrix(matriz_p, tc="d")
        
        matriz_q = np.ones((db.shape[0],1)) * (-1)
        self.q = matrix(matriz_q)
                   
        self.g = matrix(np.identity(db.shape[0]) *(-1), tc="d")
                                                                        
        matriz_h = np.zeros((db.shape[0],1))
        self.h = matrix(matriz_h) 
    
    def kernel_rbf(self, x, z, beta):
        return np.exp(-(np.sqrt(np.sum((x-z)**2))/(2*(beta**2))))

        
    def train(self, db, class_db, beta=1):
        self.initParams(db, class_db, beta)
        s = qp(self.p,self.q, self.g, self.h)
        self.alfa = s['x']

    def predict(self, linha):
        x = np.array(list(linha) + [1])
        lista_kernel = []
        for indice, linha in self.x.iterrows():
            y = self.y.iloc[indice]
            k = self.kernel_rbf(x, linha, self.beta)
            lista_kernel += [k * y]
            
        kernel = np.array(lista_kernel)         
        u = np.dot(kernel,self.alfa)
        return 1 if u > 0 else -1

In [ ]:
ksvm = KernelSVM()

In [ ]:
test_model(ksvm, iris_db, species, "setosa", [-1,1])

In [ ]:
metrics(ksvm, 20, iris_db, species, "setosa", [-1,1])

<h3 style="text-align:center;">Modelos de Regressão</h3>

In [ ]:
def test_model_regressao(modelo, db, class_db):
    train_x, test_x, train_y, test_y = train_test(db, class_db)
    modelo.train(train_x,train_y)
    teste_1 = modelo.predict(db.loc[0])
    teste_2 = modelo.predict(db.loc[30])
    teste_3 = modelo.predict(db.loc[90])
    return "dado de indice 0 = {}, 30 = {}, valor da 90 = {}".format(teste_1,teste_2,teste_3)

In [ ]:
def gerarGrafico(modelo_1, modelo_2, db, class_db):
    train_x, test_x, train_y, test_y = train_test(db, class_db)
    modelo_1.train(train_x,train_y)
    modelo_2.train(train_x,train_y)
    # Configurações do gráfico
    plt.figure(figsize=(8,4))
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Função geradora x Funções encontradas pelos modelos')
    # Plotar função gerado
    plt.scatter(db,class_db, c="#ff0000")
    for indice, linha in db.iterrows():
        y_1 = modelo_1.predict(linha)
        # Plotar função obtido do modelo 1
        plt.scatter(linha,y_1,c="#00ff00")
        y_2 = modelo_2.predict(linha)
        # Plotar função obtido do modelo 2
        plt.scatter(linha,y_2,c="#0000ff")
    plt.legend(["Função geradora", "Regressão linear", "Regressão Linear com Kernel RBF"], loc=2)
    plt.savefig("Grafico_das_funcoes_encontradas.png")

<h5>Obtendo dados</h5>

A função para obter a base de dados: f(x) = 5x + 3 + ruido

In [ ]:
def funcao_regressao(x):
    return 10*x**2 + 5 + np.random.random()

In [ ]:
regressao_class = pd.Series([funcao_regressao(x) for x in np.arange(0.0, 1.0, 0.01)])
regressao_class.loc[90]

In [ ]:
regressao_db = pd.DataFrame(np.arange(0.0, 1.0,0.01), columns=["x1"])
regressao_db.head()

In [ ]:
plt.figure(figsize=(8,4))
plt.xlabel('x')
plt.ylabel('y')
plt.title('Função geradora f(x)=10*x**2 + 5 + ruido')
# Plotar função geradora
plt.scatter(regressao_db,regressao_class, c="#ff0000")

<h5>Regressão Linear</h5>

In [ ]:
class RegressaoLinear:
    __slots__ = ['w']
    
    def __init__(self):
        self.w = []
    
    def train(self, db, class_db, epocas=100,taxa_de_aprendizagem=0.01):
        self.w = np.random.random((1,db.shape[1] + 1))
        
        x = np.concatenate((db, np.ones((db.shape[0],1)) ), axis=1)
        new_db = pd.DataFrame(x)
        new_db.index = db.index
        matriz_inverse = np.linalg.inv(np.dot(new_db.T,new_db))
        x_T_y = np.dot(new_db.T , class_db)
        self.w = np.dot(matriz_inverse, x_T_y)
        
    def predict(self, linha):
        x = np.array(list(linha) + [1])
        return np.dot(x,self.w.T)

In [ ]:
rl = RegressaoLinear()

In [ ]:
test_model_regressao(rl, regressao_db, regressao_class)

*implementar metrica para regressão*

<h5>Regressão Linear com Kernel</h5>

In [ ]:
class RegressaoLinearKernel:
    __slots__ = ['w','x_train', 'alfa']
    
    def __init__(self):
        self.w = []
        self.x_train = []
        self.alfa = []
    
    def train(self, db, class_db, epocas=100,taxa_de_aprendizagem=0.01, alfa=1):
        self.w = np.random.random((1,db.shape[1] + 1))
        self.alfa = alfa
        
        x = np.concatenate((db, np.ones((db.shape[0],1)) ), axis=1)
        new_db = pd.DataFrame(x)
        self.x_train = new_db
        kernel = []
        for i in range(new_db.shape[0]):
            linha_k = []
            for j in range(new_db.shape[0]):
                linha_k += [self.kernel_rbf(new_db.iloc[i], new_db.iloc[j], self.alfa)]
            kernel += [linha_k]
        
        matriz_kernel = np.array(kernel)
        
        matriz_inverse = np.linalg.inv(matriz_kernel)
        self.w = np.dot(matriz_inverse, class_db)
        
    def kernel_rbf(self,x,z,alfa):
        return np.exp(-(np.sqrt(np.sum((x-z)**2))/(2*(alfa**2))))
    
    def predict(self, linha):
        x = np.array(list(linha) + [1])
        linha_kernel = []
        for i in range(self.x_train.shape[0]):
            linha_kernel += [self.kernel_rbf(x,self.x_train.iloc[i], self.alfa)]
        
        kernel = np.array(linha_kernel)
        
        return np.dot(kernel,self.w.T)

In [ ]:
rlk = RegressaoLinearKernel()

In [ ]:
test_model_regressao(rlk, regressao_db, regressao_class)

Implementar o gráfico com a função geradora e as funções encontradas

In [ ]:
gerarGrafico(rl,rlk,regressao_db,regressao_class)